In [48]:
import re
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from math import floor

In [49]:
#Input the path of the Scan folder here
upperFolder = r'C:\Users\MarkeevP\Desktop\PEEM\MoSe2 on Au, Pent 20 nm, 0920\Long Term measurement_201007_175520\uncovered_FOV19_EEF50meV_6x37s_201008_010701'
folder = upperFolder+"\\Scan"

In [50]:
#replace with name of DAT File
datFile = folder + "\\uncovered_FOV19_EEF50meV_6x37s.DAT"

In [51]:
#Analyze DAT File: File Names and Number of Scans
energies = []
files = []
filePattern = r"\".*\.TIF\""
for line in open(datFile,"r"):
    if "AXIS_N =" in line:
        lineItems = line.split()
        numberOfImagesPerScan = int(lineItems[-1])
    if "TIF" in line:
        lineItems = line.split()
        energies.append(float(lineItems[0]))
        m = re.search(filePattern, line)
        files.append((m.group(0)[1:-1]))

In [52]:
#Create output folder for difference images
directory = "diffImagesCalculated"
if not os.path.exists(folder+"/"+directory):
    os.makedirs(folder+"/"+directory)

In [53]:
#read images
images = []
for f in files:
    fullName = folder+"/"+f
    try:
        im = Image.open(fullName)
        imarray = np.array(im, "int64")
        images.append(imarray)
    except IOError:
        print("Could not read all files")
        break

In [54]:
#sum images
fullScans = int(floor(len(images)/numberOfImagesPerScan))
images_summed = []
for i in range(len(images)):
    if i > numberOfImagesPerScan*fullScans-1:
        break
    if i<numberOfImagesPerScan:
        images_summed.append(images[i])
    else:
        images_summed[i%numberOfImagesPerScan] += images[i]

In [55]:
#generate and save difference images
for i in range(1,len(images_summed)):
    difImage = -(images_summed[i]-images_summed[i-1])
    dif = Image.fromarray(difImage.astype("int32"))
    dif.save(folder+"/"+directory+"/"+files[i-1][:-4]+".TIF")